In [ ]:
from flask import Flask, render_template, Response, request
from keras.models import load_model
from PIL import Image, ImageOps
import numpy as np
import cv2

app = Flask(__name__)

# Load the model and labels
model = load_model("Model/keras_Model.h5", compile=False)
class_names = open("Model/labels.txt", "r").readlines()

# Create the array of the right shape to feed into the keras model
data = np.ndarray(shape=(1, 224, 224, 3), dtype=np.float32)

# Capture image from camera
cap = cv2.VideoCapture(0) # Use the first camera
ret, frame = cap.read() # Read a frame from the camera

def gen():
    while True:
        # Read a frame from the camera
        ret, frame = cap.read()
        
        # Preprocess the image
        size = (224, 224)
        image = cv2.resize(frame, size, interpolation=cv2.INTER_AREA)
        image = Image.fromarray(image).convert("RGB")
        image = ImageOps.fit(image, size, Image.Resampling.LANCZOS)

        # turn the image into a numpy array
        image_array = np.asarray(image)

        # Normalize the image
        normalized_image_array = (image_array.astype(np.float32) / 127.5) - 1

        # Load the image into the array
        data[0] = normalized_image_array

        # Predicts the model
        prediction = model.predict(data)
        index = np.argmax(prediction)
        class_name = class_names[index]
        confidence_score = prediction[0][index]

        # Display the prediction and confidence score on the image
        text = "Prediction: {} ({:.2f}%)".format(class_name[2:], confidence_score * 100)
        cv2.putText(frame, text, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 0), 2)

        # Encode the image as a JPEG string
        ret, jpeg = cv2.imencode('.jpg', frame)
        frame_bytes = jpeg.tobytes()

        # Yield the image as a response
        yield (b'--frame\r\n'
               b'Content-Type: image/jpeg\r\n\r\n' + frame_bytes + b'\r\n')

@app.route('/')
def index():
    return render_template('new.html')

@app.route('/video_feed')
def video_feed():
    return Response(gen(),
                    mimetype='multipart/x-mixed-replace; boundary=frame')

@app.route('/quit', methods=['POST'])
def quit():
    cap.release()
    cv2.destroyAllWindows()
    return 'Quitting...'

if __name__ == '__main__':
    app.run()


 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [27/May/2023 13:19:00] "GET / HTTP/1.1" 200 -


1/1 [==============================] - 2s 2s/step


127.0.0.1 - - [27/May/2023 13:19:02] "GET /video_feed HTTP/1.1" 200 -


1/1 [==============================] - 0s 34ms/step


127.0.0.1 - - [27/May/2023 13:19:02] "GET /favicon.ico HTTP/1.1" 404 -


1/1 [==============================] - 0s 36ms/step


1/1 [==============================] - 0s 33ms/step


1/1 [==============================] - 0s 44ms/step


1/1 [==============================] - 0s 37ms/step
